In [1]:
addprocs(["heathred"])
addprocs(["heathred"])
addprocs(["heathred"])
addprocs(["heathred"])
addprocs(["amon"], dir="")
addprocs(["amon"], dir="")
addprocs(["amon"], dir="")
addprocs(["amon"], dir="")

expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error


1-element Array{Any,1}:
 9

In [2]:
ENV["LINES"] = 30
ENV["COLUMNS"] = 300

300

In [3]:
@everywhere using RecursiveAutoencoders

In [4]:
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

In [5]:
using PyCall
@pyimport nltk
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2499: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


tokenize (generic function with 1 method)

In [6]:
include("load_embeddings.jl")

load_embeddings (generic function with 1 method)

In [7]:
LL,word_indexes, indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(50,268810)
Dict{String,Int64}
Array{String,1}


In [8]:
rr = RAE(LL,word_indexes,indexed_words);

sent = "the boy destroyed the house"
sent_toks = tokenize(sent)

tree, act_tree, pp, score_total = eval_to_tree(rr,sent_toks);

In [9]:
using Iterators
@pyimport nltk.corpus as nltk_corpus
n_training = 3000
#training_sents = @pipe nltk_corpus.brown[:sents]() |> take(_,n_training)  |> collect |> convert(Vector{Vector{String}},_);
training_sents = @pipe nltk_corpus.brown[:sents]() |> filter(s->1<length(s)<=5, _) |> take(_,n_training)  |> collect |> convert(Vector{Vector{String}},_);


In [10]:
training_sents |> size

(3000,)

In [15]:
using Optim #https://github.com/JuliaOpt/Optim.jl

rae_outer = RAE(LL,word_indexes,indexed_words);

function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e = reshape(θ[1: W_e_len],W_e_shape)
    rae.b_e = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d = reshape(θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len],W_d_shape)
    rae.b_d = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    [rae.W_e[:],rae.b_e, rae.W_d[:],rae.b_d[:]] 
end

@everywhere function pack(∇W_e::Matrix{Float64}, ∇b_e::Vector{Float64}, ∇W_d::Matrix{Float64}, ∇b_d::Vector{Float64})
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end

#--------------------------------------------------------

function loss!(θ::Vector)  
    rae = unpack!(rae_outer, θ)
    @pipe training_sents |> map( ss-> eval_to_tree(rae, ss)[end], _) |> mean
    
end

function loss_grad!(θ::Vector, storage::Vector) 
    error("loss_grad! CALLED")
    storage[:] = 0
    storage = zeros(storage)
    rae = unpack!(rae_outer, θ)
    for ss in training_sents
        tree, act_tree, pp, err_total = eval_to_tree(rae, ss)
        ∇s=BPTS(rae,act_tree)
        storage+=pack(∇s...)
    end
    storage/=length(training_sents)
end

function loss_and_loss_grad!(θ::Vector, storage::Vector)   
    storage[:] = 0
    rae = unpack!(rae_outer, θ)
    err = 0.0
    #ret = @parallel (+) for ss in training_sents
    for ss in training_sents
        tree, act_tree, pp, err_total = eval_to_tree(rae, ss)
        ∇s=BPTS(rae,act_tree)
        storage[:]+=pack(∇s...)
        err+=err_total
        #[pack(∇s...), err]
    end
    #storage[:] = ret[1:end-1]
    #err=ret[end]
    
    storage[:]/=length(training_sents)
    err/=length(training_sents)
    err
end

f=DifferentiableFunction(loss!,loss_grad!,loss_and_loss_grad!)
#Must provide Graident as finite difference requires ~length(θ) calls to f
#res = optimize(f, pack(rae_outer), method=:l_bfgs, show_trace = true, store_trace = true, iterations = 2)
rae_outer = unpack!(rae_outer, res.minimum);
print("---------------------------")

LoadError: res not defined
while loading In[15], in expression starting on line 73

 - **base:** elapsed time: 138.752550969 seconds (34239757952 bytes allocated, 83.92% gc time)
 - **base 2 (after restart)**elapsed time: 232.798843691 seconds (58940288804 bytes allocated, 83.25% gc time)

In [17]:
@time res = optimize(f, pack(rae_outer), method=:l_bfgs, show_trace = true, store_trace = true, iterations = 1);

Iter     Function value   Gradient norm 
     0     6.040578e+00     9.773430e+00
     1     5.602851e+00     5.075362e+00
elapsed time: 218.871320057 seconds (55659110992 bytes allocated, 83.33% gc time)


In [ ]:
#==
    method::ASCIIString
    initial_x::Array{T,N}
    minimum::Array{T,N}
    f_minimum::Float64
    iterations::Int
    iteration_converged::Bool
    x_converged::Bool
    xtol::Float64
    f_converged::Bool
    ftol::Float64
    gr_converged::Bool
    grtol::Float64
    trace::OptimizationTrace
    f_calls::Int
    g_calls::Int
=#
@printval res.f_calls 
@printval res.g_calls 
@printval res.x_converged 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.trace


https://github.com/JuliaLang/julia/blob/master/doc/manual/profile.rst Actual instructions on profiling



In [ ]:
Profile.clear()
@profile f(pack(rae_outer))


In [ ]:
using ProfileView
ProfileView.view()

In [ ]:
function cosine_dist(a,b)
    (a⋅b)/(norm(a)*norm(b))
end

function neighbour_dists(cc::Vector{Float64}, globe::Matrix{Float64})
    [cosine_dist(cc, globe[:,ii]) for ii in 1:size(globe,2)]
end


function show_best(rae::RAE,ĉ::Embedding, nbest=20)
    candidates=neighbour_dists(ĉ,rae.L)   
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[rae.indexed_words[ii] score] for (ii,score) in best_cands]...)
end

function show_bests(rae::RAE,ĉs::Embeddings, nbest=20)
    hcat([show_best(rae,ĉs[:,ii],nbest) for ii in 1:size(ĉs,2)]...)
end


In [ ]:
sent = "run you fools"
sent_toks = tokenize(sent)

tree, act_tree, pp, score_total = eval_to_tree(rae_outer,sent_toks);
println(tree)

ĉs = unfold(rae_outer,tree,pp)
show_bests(rae_outer, ĉs)
